In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from importlib import reload 

In [2]:
from deeprank.dataset import DataLoader, PairGenerator, ListGenerator

In [3]:
loader = DataLoader('./config/letor07_mp_fold1.model')

[./data/letor/r5w/word_dict.txt]
	Word dict size: 193367
[./data/letor/r5w/qid_query.txt]
	Data size: 1692
[./data/letor/r5w/docid_doc.txt]
	Data size: 65323
[./data/letor/r5w/embed_wiki-pdc_d50_norm]
	Embedding size: 109282
Generate numpy embed: (193368, 50)


In [4]:
import json
letor_config = json.loads(open('./config/letor07_mp_fold1.model').read())

In [5]:
Letor07Path = letor_config['data_dir']

letor_config['fill_word'] = loader._PAD_
letor_config['embedding'] = loader.embedding
letor_config['feat_size'] = loader.feat_size
letor_config['vocab_size'] = loader.embedding.shape[0]
letor_config['embed_dim'] = loader.embedding.shape[1]

pair_gen = PairGenerator(rel_file=Letor07Path + '/relation.train.fold%d.txt'%(letor_config['fold']), 
                         config=letor_config)

[./data/letor/r5w/relation.train.fold1.txt]
	Instance size: 47828
Pair Instance Count: 325439


In [14]:
from deeprank import select_module
from deeprank import rank_module

select_module = reload(select_module)
rank_module = reload(rank_module)

In [15]:
select_net = select_module.IdentityNet(config=letor_config)
select_net.train()

IdentityNet()

In [16]:
letor_config['simmat_channel'] = 1
letor_config['conv_params'] = [(8, 2, 10)]
letor_config['fc_params'] = [50]
letor_config['dpool_size'] = [3, 10]
letor_config['lr'] = 0.001
letor_config['finetune_embed'] = False
rank_net = rank_module.MatchPyramidNet(config=letor_config)
rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
rank_net.train()
optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [17]:
for i in range(300):
    X1, X1_len, X2, X2_len, Y, F = pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
    X1, X2, X1_len, X2_len = select_net(X1, X2, X1_len, X2_len)
    output = rank_net(X1, X2, X1_len, X2_len)
    loss = rank_net.pair_loss(output, Y)
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor(0.9954, grad_fn=<MeanBackward1>)
tensor(0.9947, grad_fn=<MeanBackward1>)
tensor(0.9959, grad_fn=<MeanBackward1>)
tensor(1.0077, grad_fn=<MeanBackward1>)
tensor(0.9886, grad_fn=<MeanBackward1>)
tensor(0.9912, grad_fn=<MeanBackward1>)
tensor(0.9999, grad_fn=<MeanBackward1>)
tensor(0.9914, grad_fn=<MeanBackward1>)
tensor(0.9872, grad_fn=<MeanBackward1>)
tensor(0.9851, grad_fn=<MeanBackward1>)
tensor(0.9875, grad_fn=<MeanBackward1>)
tensor(0.9996, grad_fn=<MeanBackward1>)
tensor(0.9778, grad_fn=<MeanBackward1>)
tensor(0.9841, grad_fn=<MeanBackward1>)
tensor(1.0018, grad_fn=<MeanBackward1>)
tensor(0.9916, grad_fn=<MeanBackward1>)
tensor(0.9892, grad_fn=<MeanBackward1>)
tensor(0.9950, grad_fn=<MeanBackward1>)
tensor(0.9867, grad_fn=<MeanBackward1>)
tensor(0.9800, grad_fn=<MeanBackward1>)
tensor(0.9808, grad_fn=<MeanBackward1>)
tensor(0.9957, grad_fn=<MeanBackward1>)
tensor(0.9959, grad_fn=<MeanBackward1>)
tensor(0.9794, grad_fn=<MeanBackward1>)
tensor(0.9757, grad_fn=<MeanBackward1>)


tensor(0.9652, grad_fn=<MeanBackward1>)
tensor(0.9771, grad_fn=<MeanBackward1>)
tensor(0.9563, grad_fn=<MeanBackward1>)
tensor(0.9785, grad_fn=<MeanBackward1>)
tensor(0.9840, grad_fn=<MeanBackward1>)
tensor(0.9928, grad_fn=<MeanBackward1>)
tensor(0.9706, grad_fn=<MeanBackward1>)
tensor(0.9720, grad_fn=<MeanBackward1>)
tensor(0.9307, grad_fn=<MeanBackward1>)
tensor(0.9810, grad_fn=<MeanBackward1>)
tensor(0.9955, grad_fn=<MeanBackward1>)
tensor(0.9388, grad_fn=<MeanBackward1>)
tensor(0.9946, grad_fn=<MeanBackward1>)
tensor(0.9786, grad_fn=<MeanBackward1>)
tensor(0.9763, grad_fn=<MeanBackward1>)
tensor(0.9654, grad_fn=<MeanBackward1>)
tensor(0.9758, grad_fn=<MeanBackward1>)
tensor(0.9795, grad_fn=<MeanBackward1>)
tensor(0.9541, grad_fn=<MeanBackward1>)
tensor(0.9639, grad_fn=<MeanBackward1>)
tensor(0.9876, grad_fn=<MeanBackward1>)
tensor(0.9856, grad_fn=<MeanBackward1>)
tensor(0.9227, grad_fn=<MeanBackward1>)
tensor(0.9402, grad_fn=<MeanBackward1>)
tensor(0.9478, grad_fn=<MeanBackward1>)
